In [2]:
import requests
from bs4 import BeautifulSoup as soup

In [4]:
html = requests.get('https://www.tripadvisor.com/Attractions-g294229-Activities-Jakarta_Java.html')
html.status_code

<bound method Response.raise_for_status of <Response [403]>>

In [7]:
html = requests.get('https://maps.googleapis.com/maps/api/place/nearbysearch/json')
html.text

'{\n   "error_message" : "You must use an API key to authenticate each request to Google Maps Platform APIs. For additional information, please refer to http://g.co/dev/maps-no-account",\n   "html_attributions" : [],\n   "results" : [],\n   "status" : "REQUEST_DENIED"\n}'

In [ ]:
from openai import OpenAI

client = OpenAI(api_key='sk-proj-RaPQwENoFDL6ohssuHd6T3BlbkFJBMXifkXqdxRwp4iXFWHA')

def recommend_attractions(prompt):
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "user", "content": prompt}
            ],
            max_tokens=200,
            temperature=0.7,
            n=1,
            stop=None,
            stream=True
        )
        
        output = ""
        
        for chunk in response:
            if chunk.choices[0].delta.content is not None:
                output += chunk.choices[0].delta.content
        
        return output.strip()
        
    except Exception as e:
        return f"An error occurred: {str(e)}"

user_prompt = "I am looking for family-friendly attractions near here with good food options nearby."
recommendations = recommend_attractions(user_prompt)
print("Recommended Attractions:\n", recommendations)


Recommended Attractions:
 Here are some family-friendly attractions near here with good food options nearby:

1. Adventure Park: A fun-filled amusement park with rides and attractions for all ages. Nearby, you can find a variety of food options such as fast food, pizza, and ice cream.

2. Zoo: A popular zoo with a wide range of animals to see. There are several cafes and restaurants nearby offering kid-friendly meals and snacks.

3. Water Park: A water park with slides, pools, and splash pads. There are food stands and snack bars located throughout the park serving tasty treats and meals.

4. Miniature Golf Course: A mini-golf course with challenging holes and obstacles. Nearby, you can find restaurants and cafes serving delicious meals and desserts.

5. Botanical Garden: A beautiful botanical garden with colorful flowers and plants. There are picnic areas where you can enjoy a meal or snack from nearby restaurants and food trucks.


In [5]:
bsobj = soup(html.content, 'lxml')

In [6]:
# <div class="XfVdV o YQJrd"><span class="XQCOK">2.</span> Grand Indonesia Mall</div>
places = []
for name in bsobj.findAll('div', {'class': 'XfVdV o YQJrd'}):
    places.append(name.text.strip())
    
places

[]

In [ ]:
# <svg class="UctUV d H0 hzzSG" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-rrn:"><title id=":lithium-rrn:">4.5 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg>
# <title id=":lithium-rrn:">4.5 of 5 bubbles</title>
ratings = []
for r in bsobj.findAll('svg', {'class': 'UctUV d H0 hzzSG'}):
    title_element = r.find('title')
    if title_element:
        ratings.append(title_element.text)
ratings

In [ ]:
reviews = []
for rev in bsobj.findAll('span', {'class': 'biGQs _P pZUbB hmDzD'}):
    reviews.append(rev.title.strip())
    
reviews

In [ ]:
category = []
for c in bsobj.findAll('span', {'class': 'biGQs _P pZUbB avBIb hmDzD'}):
    category.append(name.text.strip())
category

In [ ]:
import pandas as pd
d1 = {'Place': places, 'Ratings': ratings, 'Number of Reviews': reviews, 'Category': category}
df = pd.DataFrame.from_dict(d1)

df